In [1]:
import numpy as np
import math
import pandas as pd
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison


In [2]:
from bokeh.io import show
from bokeh.plotting import figure
#from  big_data_tools.bokeh_tools.bar import bar 
#from  big_data_tools.bokeh_tools.histogram import hist 
#from  big_data_tools.bokeh_tools.box_plot import box_plot 
#from  big_data_tools.bokeh_tools.prob_plot import prob_plot 
from bokeh.layouts import gridplot
from bokeh.models import NumeralTickFormatter
from bokeh.models import DatetimeTickFormatter

In [3]:
from statsmodels.graphics.factorplots import interaction_plot


In [4]:
def get_data():
    path = '/home/henry/Downloads/ToothGrowth.csv'
    df = data = pd.read_csv(path)
    return df

df = get_data()
df.head()
    

,Unnamed: 0,len,supp,dose
0,1,4.2,VC,0.5
1,2,11.5,VC,0.5
2,3,7.3,VC,0.5
3,4,5.8,VC,0.5
4,5,6.4,VC,0.5


In [5]:
df2 = df.groupby('dose')
df2.mean()['len'].index
del(df2)

In [6]:
def get_interaction_plot(df):
    grouped = df.groupby(['supp', 'dose'])
    y1 = list(grouped.mean()['len']['OJ'])
    y2 = list(grouped.mean()['len']['VC'])
    x = df['dose'].unique()
    p = figure()
    p.line(x = x, y = y1)
    p.line(x = x, y = y2)
    return p
p_interaction = get_interaction_plot(df)


In [7]:
def get_degrees_freedom(data):
    print(type(data))
    N = len(data.len)
    df_a = len(data.supp.unique()) - 1
    df_b = len(data.dose.unique()) - 1
    df_axb = df_a*df_b 
    df_w = N - (len(data.supp.unique())*len(data.dose.unique()))
    return df_a, df_b, df_axb, df_w
df_a, df_b, df_axb, df_w = get_degrees_freedom(df)

<class 'pandas.core.frame.DataFrame'>


In [8]:
def get_sum_squares(data):
    grand_mean = data['len'].mean()
    ssq_a = sum([(data[data.supp ==l].len.mean()-grand_mean)**2 for l in data.supp])
    ssq_b = sum([(data[data.dose ==l].len.mean()-grand_mean)**2 for l in data.dose])
    ssq_t = sum((data.len - grand_mean)**2)
    vc = data[data.supp == 'VC']
    oj = data[data.supp == 'OJ']
    vc_dose_means = [vc[vc.dose == d].len.mean() for d in vc.dose]
    oj_dose_means = [oj[oj.dose == d].len.mean() for d in oj.dose]
    ssq_w = sum((oj.len - oj_dose_means)**2) +sum((vc.len - vc_dose_means)**2)
    ssq_axb = ssq_t-ssq_a-ssq_b-ssq_w
    #my guess here
    sse = ssq_w + ssq_axb
    return ssq_t, ssq_a, ssq_b, ssq_w, ssq_axb, sse

ssq_t, ssq_a, ssq_b, ssq_w, ssq_axb, sse = get_sum_squares(df)

In [9]:
def get_f_scores(ssq_a,ssq_b, ssq_axb, ssq_w, df_a, df_b, df_axb, df_w):
    ms_a = ssq_a/df_a
    ms_b = ssq_b/df_b
    ms_axb = ssq_axb/df_axb
    ms_w = ssq_w/df_w
    f_a = ms_a/ms_w
    f_b = ms_b/ms_w
    f_axb = ms_axb/ms_w
    return {'f_a':f_a, 'f_b':f_b, 'f_axb':f_axb}
d_fscores = get_f_scores(ssq_a = ssq_a, ssq_b = ssq_b, ssq_axb = ssq_axb, ssq_w = ssq_w,
             df_a = df_a, df_b = df_b, df_axb = df_axb, df_w = df_w
)

In [10]:
def get_p_values(f_a, f_b, f_axb):
    p_a = stats.f.sf(f_a, df_a, df_w)
    p_b = stats.f.sf(f_b, df_b, df_w)
    p_axb = stats.f.sf(f_axb, df_axb, df_w)
    return p_a, p_b, p_axb
get_p_values(f_a = d_fscores['f_a'] , f_b = d_fscores['f_b'], f_axb = d_fscores['f_axb'])

(0.00023118280977342404, 4.046291195992212e-18, 0.021860268964791103)

In [11]:
def func_a():
    return {'a':1, 'b':2}
def func_b():
    return {'c':3}
def func_c(a, b, c):
    print(a, b, c)
def func_d():
    return 4, 5, 6
a, b = func_a()
func_c(**func_a(), **func_b())
func_d()
func_c(*func_d())

1 2 3
4 5 6


In [12]:
grid = gridplot([p_interaction], ncols=2)
show(grid)